# 1) Setup
## Modules

In [1]:
# Libraries
import pandas as pd

## Data

In [2]:
# Import data
df_business = pd.read_parquet('data/ATML2024_businesses.parquet')

df_Train_reviews = pd.read_parquet('data/ATML2024_reviews_train.parquet')

df_users = pd.read_parquet('data/ATML2024_users.parquet')

In [9]:
# General information on the datasets
# USER DATASET
df_users.info()
df_users.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747468 entries, 0 to 747467
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             747468 non-null  object 
 1   name                747457 non-null  object 
 2   user_since          747468 non-null  object 
 3   useful              747468 non-null  float64
 4   funny               747468 non-null  float64
 5   cool                747468 non-null  float64
 6   premium_account     57420 non-null   object 
 7   friends             747468 non-null  float64
 8   fans                747468 non-null  float64
 9   compliment_hot      747468 non-null  float64
 10  compliment_more     747468 non-null  float64
 11  compliment_profile  747468 non-null  float64
 12  compliment_cute     747468 non-null  float64
 13  compliment_list     747468 non-null  float64
 14  compliment_note     747468 non-null  float64
 15  compliment_plain    747468 non-nul

,useful,funny,cool,friends,fans,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer
count,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000,747468.00000,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000,747468.000000
mean,74.840044,31.100846,45.027212,61.342055,2.528844,3.388153,0.527842,0.350840,0.24852,0.130927,2.688904,5.711291,5.290550,5.290550,1.992893
std,965.774055,605.662158,849.247796,178.325034,27.658841,107.391836,19.975968,23.747047,18.05827,16.143172,94.760818,176.622435,137.359405,137.359405,47.156571
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.000000,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,21.000000,4.000000,5.000000,56.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,206296.000000,185823.000000,199878.000000,14995.000000,12497.000000,25784.000000,13501.000000,14180.000000,13654.00000,12669.000000,59031.000000,101097.000000,49967.000000,49967.000000,15934.000000


In [10]:
# REVIEWS DATASET
df_Train_reviews.info()
df_Train_reviews.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050000 entries, 0 to 1049999
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           1050000 non-null  int64 
 1   user_id      1050000 non-null  object
 2   business_id  1050000 non-null  object
 3   rating       1050000 non-null  int64 
 4   useful       1050000 non-null  int64 
 5   funny        1050000 non-null  int64 
 6   cool         1050000 non-null  int64 
 7   text         1050000 non-null  object
 8   date         1050000 non-null  object
dtypes: int64(5), object(4)
memory usage: 72.1+ MB


,id,rating,useful,funny,cool
count,1.050000e+06,1.050000e+06,1.050000e+06,1.050000e+06,1.050000e+06
mean,5.249995e+05,3.746190e+00,1.183533e+00,3.266943e-01,4.975705e-01
std,3.031090e+05,1.479862e+00,3.298671e+00,1.671890e+00,2.192185e+00
min,0.000000e+00,1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,2.624998e+05,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.249995e+05,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.874992e+05,5.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.049999e+06,5.000000e+00,9.330000e+02,3.750000e+02,3.760000e+02


In [11]:
# BUSINESS DATASET
df_business.info()
df_business.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138210 entries, 0 to 138209
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  138210 non-null  object 
 1   name         138210 non-null  object 
 2   address      133772 non-null  object 
 3   city         138210 non-null  object 
 4   state        138210 non-null  object 
 5   postal_code  138145 non-null  object 
 6   latitude     138210 non-null  float64
 7   longitude    138210 non-null  float64
 8   attributes   126589 non-null  object 
 9   categories   138136 non-null  object 
 10  hours        117852 non-null  object 
dtypes: float64(2), object(9)
memory usage: 11.6+ MB


,latitude,longitude
count,138210.000000,138210.000000
mean,36.648027,-89.363649
std,5.841371,14.928374
min,27.555127,-120.095137
25%,32.187324,-90.355706
50%,38.772051,-86.121655
75%,39.953487,-75.419145
max,53.679197,-73.200457


### Transform data
Possibilities:
- Log when data is highly skewed.
- Binning of continuous variables.
- Simplifying of categories: Trying to aggregate to higher-level categories when it makes sense.

## Data quality assessment and profiling

1. Categorical: 
	- count
	- count distinct
	- assess unique values
2. Numerical: 
	- count
	- min
	- max

Do we have missing data ?
Are there variables that are numerical but really should be categorical?

# Exploratory Data Analysis

## 1. Exploring each individual variable

Quantify:

- _Location_:
	- mean
	- median
	- mode
	- interquartile mean
- _Spread_:
	- standard deviation
	- variance
	- range
	- interquartile range
- _Shape_:
	- skewness
	- kurtosis

For time series:
	plot summary statistics over time.

For panel data:
- Plot cross-sectional summary statistics over time
- Plot time-series statistics across the population

#### Question
- What does each field in the data look like?
    - Is the distribution skewed? Bimodal?
    - Are there outliers? Are they feasible?
    - Are there discontinuities?
- Are the typical assumptions seen in modeling valid?
    - Gaussian
    - Identically and independently distributed
    - Have one mode
    - Can be negative
    - Generating processes are stationary and isoptropic (time series)
    - Independence between subjects (panel data)

## 2. Exploring the relationship between each variable and the target

How does each field interact with the target?

Assess each relationship’s:
- Linearity
- Direction
- Rough size
- Strength
Methods:
- Bivariate visualizations
- Calculate correlation

## 3. Assessing interactions between variables

How do the variables interact with each other?
- Bivariate visualizations
- Correlation matrices
- Compare summary statistics of variable x for different categories of y


## 4. Exploring data across many dimensions

- Categorical:
    - Parallel coordinates
- Continuous
    - Principal component analysis
    - Clustering